In [ ]:
# Jason Hemedinger
# Argonne National Laboratory

In [ ]:
import pyart, boto3, tempfile, os, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from botocore.handlers import disable_signing
from tint import Cell_tracks
from tint import animate as tint_animate
from tint.visualization import embed_mp4_as_gif
from glob import glob
from datetime import datetime
from pylab import *

In [ ]:
def get_current_scan(station, key_index):
    '''
    Function will pull the latest radar scan from any radar site using 
    Amazon S3.
    ----------
    station = Four letter NEXRAD identifier, must be entered using quote marks
              Example: 'KEPZ'
            
    key_index = Number of keys you want pulled. 
                If a positive number then the number of keys pulled will start from the first key of the day.
                If a negative number then the number of keys pulled will count backwards from most recent key.
                Example: 15 would pull the first 15 keys for the day, while -15 would
                pull the 15 most recent keys
    '''
    #creating a bucket and a client to be able to pull data from AWS and setting 
    #it as unsigned
    bucket = 'noaa-nexrad-level2'
    s3 = boto3.resource('s3')
    s3.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
    
    #connects the bucket create above with radar data
    aws_radar = s3.Bucket(bucket)
    
    #setting the date and time to current.
    #this will allow for allow the current date's radar scands to be pulled
    desired_datetime = datetime.utcnow()
    target_string = datetime.strftime(desired_datetime, '%Y/%m/%d/'+station)
    
    for obj in aws_radar.objects.filter(Prefix= target_string):
        '{0}:{1}'.format(aws_radar.name, obj.key)
    my_list_of_keys = [this_object.key for this_object in aws_radar.objects.filter(Prefix= target_string)]
    keys = my_list_of_keys[key_index:]
    print(my_list_of_keys[key_index:])
    
    return aws_radar, keys

In [ ]:
#creating a new directory for each day of an event
path = '/home/jhemedinger/suli_projects/chicago-nowcast/events'
date = datetime.utcnow().strftime('%Y_%m_%d')
date = str(date)
out_dir_path = path+'/'+date
out_dir = os.makedirs(out_dir_path, exist_ok=True)

#within the newly created directory, creating a directory for each event
event_date = datetime.utcnow().strftime('%Y%m%d-%H')
event_date = str(event_date)
out_path_dir = path+'/'+date+'/'+event_date+'Z'
out_path = os.makedirs(out_path_dir, exist_ok=True)

In [ ]:
#setting the radar site and which keys to pull.
aws_radar, keys = get_current_scan('KLOT', key_index=-10)

In [ ]:
def animate(nframe):
    plt.clf()
    localfile = tempfile.NamedTemporaryFile()
    aws_radar.download_file(keys[nframe], localfile.name)
    # Only pulling two scans for the sake of time and memory.
    radar = pyart.io.read(localfile.name)
    display = pyart.graph.RadarMapDisplay(radar)
    # Delete radar after use to save memory.
    del radar
    display.plot_ppi_map('reflectivity', resolution='l', 
                         vmin=-8, vmax=64, mask_outside=False,
                         sweep=0, width=350000, height=350000,
                        cmap=pyart.graph.cm.LangRinbow12)
    display.basemap.drawcounties()
    display.plot_point(-87.981810, 41.713969, label_text='ANL', color='k')
fig = plt.figure(figsize=(10, 8))
anim_klot = animation.FuncAnimation(fig, animate,
                                    frames=len(keys))
anim_klot.save(out_path_dir+'/reflectivity_animation.gif', 
               writer='imagemagick', fps=2)
plt.show()
plt.close()

In [ ]:
#turing the data into grid data and saving it to a folder
def get_grid(aws_radar, key):
    localfile = tempfile.NamedTemporaryFile()
    aws_radar.download_file(key, localfile.name)
    radar = pyart.io.read(localfile.name)
    grid = pyart.map.grid_from_radars(
            radar, grid_shape=(31, 401, 401),
            grid_limits=((0, 15000), (-200000, 200000), (-200000, 200000)),
            fields=['reflectivity'], gridding_algo='map_gates_to_grid',
            h_factor=0., nb=0.6, bsp=1., min_radius=200.)
    return grid

for num,key in enumerate(keys):
    print('saving grid', num)
    grid = get_grid(aws_radar, key)
    name = os.path.join(out_path_dir, 'grid_' + str(num).zfill(3) + '.nc')
    pyart.io.write_grid(name, grid)
    del grid

In [ ]:
#files_2 = [os.path.join(out_dir, fn) for fn in os.listdir(out_dir)]
files = glob(out_path_dir + '/grid_*')
files.sort()

In [ ]:
grid_gen = (pyart.io.read_grid(f) for f in files)
#grid_list = [pyart.io.read_grid(f) for f in files]

In [ ]:
tracks_obj = Cell_tracks()
tracks_obj.params

In [ ]:
tracks_obj.params['FIELD_THRESH']=35

In [ ]:
tracks_obj.get_tracks(grid_gen)

In [ ]:
tracks_obj.tracks

In [ ]:
if os.path.exists(out_path_dir + '/test_animation.mp4'):
    print(out_path_dir + '/test_animation.mp4'
          + ' already exists, removing file')
    os.remove(out_path_dir + '/test_animation.mp4')

In [ ]:
grid_gen = (pyart.io.read_grid(f) for f in files)
tint_animate(tracks_obj, grid_gen, os.path.join(out_path_dir, 'test_animation'))

In [ ]:
embed_mp4_as_gif(out_path_dir + '/test_animation.mp4')

In [ ]:
cells = tracks_obj.tracks.groupby(level='uid')
for uid in cells:
    print(uid)

In [ ]:
tracks_obj.tracks.groupby(level='uid').size().sort_values(ascending=False)[:]

In [ ]:
df_193 = pd.DataFrame(tracks_obj.tracks.xs('193', level='uid'))
lons, lats = np.array(df_193['lon']), np.array(df_193['lat'])
time = np.array(pd.to_datetime(df_193['time']))

In [ ]:
fit = polyfit(lon_f10,lat_f10,1)
fit_fn = poly1d(fit)

fig = plt.figure(figsize=(10,8))
plt.plot(lons, fit_fn(lons), '--b')
plt.plot(lons, lats, color='k', marker='o')
#plt.plot(-87.981810, 41.713969, color='r', marker='*')
plt.xlabel('LONGITUDE')
plt.ylabel('LATITUDE')
plt.show()
plt.close()

In [ ]:
t = (time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
x, y = lats, lons

fit_lat = polyfit(t,x,1)
fit_lon = polyfit(t,y,1)
fit_fn_lon = poly1d(fit_lon)
fit_fn_lat = poly1d(fit_lat)

fig = plt.figure(figsize=(10,8))

plt.plot(t, x, 'ro', time, fit_fn_lat(t), '--k')
plt.xlabel('TIME (UTC)')
plt.ylabel('LATITUDE')
plt.show()
plt.close()

fig = plt.figure(figsize=(10,8))
plt.plot(t, y, 'bo', t, fit_fn_lon(t), '--k')
plt.xlabel('TIME (UTC)')
plt.ylabel('LONGITUDE')
plt.show()
plt.close()

In [ ]:
def lats_lons(beginning, end, interval):
    '''
    Will predict lat/lon for a given time interval.
    Returns time, lat, and lon
    beginning: beginning of the time interval
    end: end of interval
    interval: time interval in minutes
    
    Ex: lats_lons(10, 70, 10) will find the lat/lon 
    for the next hour every 10 minutes.
    '''
    minimum = minimum
    maximum = maximum
    interval = interval
    arr = np.arange(minimum, maximum, interval) 
    my_time = []
    for i in arr:
        my_time.append(time[-1] + np.timedelta64(str(i), 'm'))
    my_new_time = np.array(my_time)
    nts = ((my_new_time - np.datetime64('1970-01-01T00:00:00Z')) 
           / np.timedelta64(1, 's'))
    my_new_lat = fit_fn_lat(nts)
    my_new_lon = fit_fn_lon(nts)
    print(my_new_time)
    print(my_new_lon)
    print(my_new_lat)
    return my_new_time, my_new_lat, my_new_lon

In [ ]:
my_new_time, my_new_lat, my_new_lon = lats_lons(10,110,10)

In [ ]:
#animating using matplotlib and pyart
def animate(nframe):
    plt.clf()
    localfile = tempfile.NamedTemporaryFile()
    aws_radar.download_file(keys[nframe], localfile.name)
    radar = pyart.io.read(localfile.name)
    display = pyart.graph.RadarMapDisplay(radar)
    # Delete radar after use to save memory.
    del radar
    display.plot_ppi_map('reflectivity', sweep=0, resolution='l',
                         vmin=-8, vmax=64, mask_outside=True, 
                         fig=fig, width=200000, height=200000,
                         cmap=pyart.graph.cm.LangRainbow12)
    display.basemap.drawcounties()
    display.plot_line_geo(my_new_lon, my_new_lat, '--r')
    display.plot_line_geo(lons[:nframe], lats[:nframe], '-k')
    display.plot_point(-87.981810, 41.713969 , label_text='ANL', symbol='ko')
fig = plt.figure(figsize=(10, 8))

anim_klot = animation.FuncAnimation(fig, animate, 
                                    frames=len(keys))
anim_klot.save(out_path_dir + '/reflectivity_animation4.gif', 
               writer='imagemagick', fps=1)
plt.show()
plt.close()